In [23]:
#Задача 1
import sqlite3

# Создаем соединение с нашей базой данных
# В нашем примере база данных будет в памяти компьютера
conn = sqlite3.connect(':memory:')

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor = conn.cursor()

# Создаем таблицу Students
cursor.execute("""
CREATE TABLE Students (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    surname TEXT NOT NULL,
    age INTEGER NOT NULL,
    city TEXT NOT NULL
)
""")

# Создаем таблицу Courses
cursor.execute("""
CREATE TABLE Courses (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    time_start TEXT NOT NULL,
    time_end TEXT NOT NULL
)
""")

# Создаем таблицу Student_courses
cursor.execute("""
CREATE TABLE Student_courses (
    student_id INTEGER,
    course_id INTEGER,
    FOREIGN KEY(student_id) REFERENCES Students(id),
    FOREIGN KEY(course_id) REFERENCES Courses(id)
)
""")

# Сохраняем изменения
conn.commit()



In [24]:
#Задача 2
# Добавляем данные в таблицу Courses
courses = [(1, 'python', '21.07.21', '21.08.21'),
           (2, 'java', '13.07.21', '16.08.21')]
cursor.executemany("""
INSERT INTO Courses VALUES (?, ?, ?, ?)
""", courses)

# Добавляем данные в таблицу Students
students = [(1, 'Max', 'Brooks', 24, 'Spb'),
            (2, 'John', 'Stones', 15, 'Spb'),
            (3, 'Andy', 'Wings', 45, 'Manhester'),
            (4, 'Kate', 'Brooks', 34, 'Spb')]
cursor.executemany("""
INSERT INTO Students VALUES (?, ?, ?, ?, ?)
""", students)

# Добавляем данные в таблицу Student_courses
student_courses = [(1, 1),
                   (2, 1),
                   (3, 1),
                   (4, 2)]
cursor.executemany("""
INSERT INTO Student_courses VALUES (?, ?)
""", student_courses)

# Сохраняем изменения
conn.commit()

# Запросы
# 1. Все студенты старше 30 лет
cursor.execute("""
SELECT * FROM Students WHERE age > 30
""")
print("Студенты старше 30 лет:")
for row in cursor.fetchall():
    print(row)

# 2. Все студенты, которые проходят курс по python
cursor.execute("""
SELECT Students.* FROM Students
JOIN Student_courses ON Students.id = Student_courses.student_id
JOIN Courses ON Student_courses.course_id = Courses.id
WHERE Courses.name = 'python'
""")
print("\nСтуденты, которые проходят курс по python:")
for row in cursor.fetchall():
    print(row)

# 3. Все студенты, которые проходят курс по python и из Spb
cursor.execute("""
SELECT Students.* FROM Students
JOIN Student_courses ON Students.id = Student_courses.student_id
JOIN Courses ON Student_courses.course_id = Courses.id
WHERE Courses.name = 'python' AND Students.city = 'Spb'
""")
print("\nСтуденты, которые проходят курс по python и из Spb:")
for row in cursor.fetchall():
    print(row)

# Закрываем соединение с базой данных
conn.close()

Студенты старше 30 лет:
(3, 'Andy', 'Wings', 45, 'Manhester')
(4, 'Kate', 'Brooks', 34, 'Spb')

Студенты, которые проходят курс по python:
(1, 'Max', 'Brooks', 24, 'Spb')
(2, 'John', 'Stones', 15, 'Spb')
(3, 'Andy', 'Wings', 45, 'Manhester')

Студенты, которые проходят курс по python и из Spb:
(1, 'Max', 'Brooks', 24, 'Spb')
(2, 'John', 'Stones', 15, 'Spb')


In [18]:
class Database:
    def __init__(self, db_name):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()

    def create_tables(self):
        self.cursor.execute("""
        CREATE TABLE Students (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            surname TEXT NOT NULL,
            age INTEGER NOT NULL,
            city TEXT NOT NULL
        )
        """)

        self.cursor.execute("""
        CREATE TABLE Courses (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            time_start TEXT NOT NULL,
            time_end TEXT NOT NULL
        )
        """)

        self.cursor.execute("""
        CREATE TABLE Student_courses (
            student_id INTEGER,
            course_id INTEGER,
            FOREIGN KEY(student_id) REFERENCES Students(id),
            FOREIGN KEY(course_id) REFERENCES Courses(id)
        )
        """)

        self.conn.commit()

    def add_data(self):
        courses = [(1, 'python', '21.07.21', '21.08.21'),
                   (2, 'java', '13.07.21', '16.08.21')]
        self.cursor.executemany("""
        INSERT INTO Courses VALUES (?, ?, ?, ?)
        """, courses)

        students = [(1, 'Max', 'Brooks', 24, 'Spb'),
                    (2, 'John', 'Stones', 15, 'Spb'),
                    (3, 'Andy', 'Wings', 45, 'Manhester'),
                    (4, 'Kate', 'Brooks', 34, 'Spb')]
        self.cursor.executemany("""
        INSERT INTO Students VALUES (?, ?, ?, ?, ?)
        """, students)

        student_courses = [(1, 1),
                           (2, 1),
                           (3, 1),
                           (4, 2)]
        self.cursor.executemany("""
        INSERT INTO Student_courses VALUES (?, ?)
        """, student_courses)

        self.conn.commit()

    def execute_query(self, query):
        self.cursor.execute(query)
        return self.cursor.fetchall()

    def close(self):
        self.conn.close()


# Тестирование функционала
db = Database(':memory:')
db.create_tables()
db.add_data()

# Проверка добавления данных
assert db.execute_query("SELECT * FROM Courses") == [(1, 'python', '21.07.21', '21.08.21'), (2, 'java', '13.07.21', '16.08.21')]
assert db.execute_query("SELECT * FROM Students") == [(1, 'Max', 'Brooks', 24, 'Spb'), (2, 'John', 'Stones', 15, 'Spb'), (3, 'Andy', 'Wings', 45, 'Manhester'), (4, 'Kate', 'Brooks', 34, 'Spb')]
assert db.execute_query("SELECT * FROM Student_courses") == [(1, 1), (2, 1), (3, 1), (4, 2)]

# Проверка выполнения произвольных запросов
assert db.execute_query("SELECT * FROM Students WHERE age > 30") == [(3, 'Andy', 'Wings', 45, 'Manhester'), (4, 'Kate', 'Brooks', 34, 'Spb')]

db.close()
